In [1]:
import numpy as np

GRID_SIZE = 4
GOAL_STATE = (3, 3)
START_STATE = (0, 0)
ACTIONS = ['U', 'D', 'L', 'R']

REWARD_GOAL = 1
REWARD_DEFAULT = 0
REWARD_INVALID = -1

In [2]:
def get_next_state(state, action):
    x, y = state
    if action == 'U':
        return (max(x - 1, 0), y)
    if action == 'D':
        return (min(x + 1, GRID_SIZE - 1), y)
    if action == 'L':
        return (x, max(y - 1, 0))
    if action == 'R':
        return (x, min(y + 1, GRID_SIZE - 1))

In [3]:
def get_reward(state):
    if state == GOAL_STATE:
        return REWARD_GOAL
    return REWARD_DEFAULT

In [4]:
def value_iteration(gamma=0.9, threshold=1e-6):
    V = np.zeros((GRID_SIZE, GRID_SIZE))
    delta = float('inf')
    while delta > threshold:
        delta = 0
        for x in range(GRID_SIZE):
            for y in range(GRID_SIZE):
                state = (x, y)
                if state == GOAL_STATE:
                    continue
                v = V[x, y]
                max_value = float('-inf')
                for action in ACTIONS:
                    next_state = get_next_state(state, action)
                    reward = get_reward(next_state)
                    value = reward + gamma * V[next_state]
                    max_value = max(max_value, value)
                V[x, y] = max_value
                delta = max(delta, abs(v - V[x, y]))
    return V

In [5]:
def policy_iteration(gamma=0.9, threshold=1e-6):
    V = np.zeros((GRID_SIZE, GRID_SIZE))
    policy = {state: np.random.choice(ACTIONS) for state in [(x, y) for x in range(GRID_SIZE) for y in range(GRID_SIZE)]}
    policy_stable = False
    while not policy_stable:
        while True:
            delta = 0
            new_V = np.copy(V)
            for x in range(GRID_SIZE):
                for y in range(GRID_SIZE):
                    state = (x, y)
                    if state == GOAL_STATE:
                        continue
                    action = policy[state]
                    next_state = get_next_state(state, action)
                    reward = get_reward(next_state)
                    new_V[x, y] = reward + gamma * V[next_state]
                    delta = max(delta, np.abs(V[x, y] - new_V[x, y]))
            V = np.copy(new_V)
            if delta < threshold:
                break
        policy_stable = True
        for x in range(GRID_SIZE):
            for y in range(GRID_SIZE):
                state = (x, y)
                if state == GOAL_STATE:
                    continue
                old_action = policy[state]
                max_value = float('-inf')
                best_action = None
                for action in ACTIONS:
                    next_state = get_next_state(state, action)
                    reward = get_reward(next_state)
                    value = reward + gamma * V[next_state]
                    if value > max_value:
                        max_value = value
                        best_action = action
                policy[state] = best_action
                if old_action != best_action:
                    policy_stable = False
    return V, policy

In [6]:
print("Running Value Iteration...")
V_value_iter = value_iteration()
print("Value Function after Value Iteration:")
print(V_value_iter)

print("\nRunning Policy Iteration...")
V_policy_iter, policy_iter = policy_iteration()

print("\nValue Function after Policy Iteration:")
print(V_policy_iter)

Running Value Iteration...
Value Function after Value Iteration:
[[0.59049 0.6561  0.729   0.81   ]
 [0.6561  0.729   0.81    0.9    ]
 [0.729   0.81    0.9     1.     ]
 [0.81    0.9     1.      0.     ]]

Running Policy Iteration...

Value Function after Policy Iteration:
[[0.59049 0.6561  0.729   0.81   ]
 [0.6561  0.729   0.81    0.9    ]
 [0.729   0.81    0.9     1.     ]
 [0.81    0.9     1.      0.     ]]


In [7]:
print("\nOptimal Policy after Policy Iteration:")
for x in range(GRID_SIZE):
    for y in range(GRID_SIZE):
        state = (x, y)
        if state == GOAL_STATE:
            print(f"({x}, {y}) -> Goal")
        else:
            print(f"({x}, {y}) -> {policy_iter[state]}")


Optimal Policy after Policy Iteration:
(0, 0) -> D
(0, 1) -> D
(0, 2) -> D
(0, 3) -> D
(1, 0) -> D
(1, 1) -> D
(1, 2) -> D
(1, 3) -> D
(2, 0) -> D
(2, 1) -> D
(2, 2) -> D
(2, 3) -> D
(3, 0) -> R
(3, 1) -> R
(3, 2) -> R
(3, 3) -> Goal
